In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1041841,2021-02-28,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1041842,2021-03-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1041843,2021-03-02,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1041844,2021-03-03,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
28231,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28233,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28235,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28237,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28239,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
645044,2021-02-28,Arkansas,Arkansas,5001,1993,33.00,5000,Arkansas,AR,Arkansas,State,3017804
645046,2021-03-01,Arkansas,Arkansas,5001,1993,33.00,5000,Arkansas,AR,Arkansas,State,3017804
645048,2021-03-02,Arkansas,Arkansas,5001,1996,33.00,5000,Arkansas,AR,Arkansas,State,3017804
645050,2021-03-03,Arkansas,Arkansas,5001,1997,33.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1041841,2021-02-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1041842,2021-03-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1041843,2021-03-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1041844,2021-03-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-04') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
408,2021-03-04,Snohomish,Washington,31182,546.00,53061,WA,County,822083,3793.05,66.42
814,2021-03-04,Cook,Illinois,476835,9972.00,17031,IL,County,5150233,9258.51,193.62
1219,2021-03-04,Orange,California,261976,4013.00,6059,CA,County,3175692,8249.41,126.37
1623,2021-03-04,Maricopa,Arizona,513035,9223.00,4013,AZ,County,4485414,11437.85,205.62
2027,2021-03-04,Los Angeles,California,1198737,21778.00,6037,CA,County,10039107,11940.67,216.93
...,...,...,...,...,...,...,...,...,...,...,...
1041397,2021-03-04,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1041540,2021-03-04,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1041652,2021-03-04,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1041760,2021-03-04,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
408,2021-03-04,Snohomish,Washington,31182,546.00,53061,WA,County,822083,3793.05,66.42,66.42,3793.05
814,2021-03-04,Cook,Illinois,476835,9972.00,17031,IL,County,5150233,9258.51,193.62,193.62,9258.51
1219,2021-03-04,Orange,California,261976,4013.00,6059,CA,County,3175692,8249.41,126.37,126.37,8249.41
1623,2021-03-04,Maricopa,Arizona,513035,9223.00,4013,AZ,County,4485414,11437.85,205.62,205.62,11437.85
2027,2021-03-04,Los Angeles,California,1198737,21778.00,6037,CA,County,10039107,11940.67,216.93,216.93,11940.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041397,2021-03-04,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1041540,2021-03-04,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1041652,2021-03-04,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1041760,2021-03-04,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
733795,2021-03-04,Clarke,Alabama,3440,53.00,1025,AL,County,23622,14562.70,224.37,224.37,14562.70,1
674375,2021-03-04,Hale,Alabama,2122,69.00,1065,AL,County,14651,14483.65,470.96,470.96,14483.65,2
528081,2021-03-04,Lowndes,Alabama,1327,51.00,1085,AL,County,9726,13643.84,524.37,524.37,13643.84,3
468402,2021-03-04,Etowah,Alabama,13390,325.00,1055,AL,County,102268,13093.05,317.79,317.79,13093.05,4
393246,2021-03-04,Franklin,Alabama,3998,78.00,1059,AL,County,31362,12747.91,248.71,248.71,12747.91,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643687,2021-03-04,Converse,Wyoming,970,16.00,56009,WY,County,13822,7017.80,115.76,115.76,7017.80,19
644029,2021-03-04,Sublette,Wyoming,679,7.00,56035,WY,County,9831,6906.72,71.20,71.20,6906.72,20
889238,2021-03-04,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,21
799512,2021-03-04,Lincoln,Wyoming,1244,11.00,56023,WY,County,19830,6273.32,55.47,55.47,6273.32,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
528081,2021-03-04,Lowndes,Alabama,1327,51.00,1085,AL,County,9726,13643.84,524.37,524.37,13643.84,3,1
674375,2021-03-04,Hale,Alabama,2122,69.00,1065,AL,County,14651,14483.65,470.96,470.96,14483.65,2,2
467022,2021-03-04,Clay,Alabama,1459,54.00,1027,AL,County,13235,11023.80,408.01,408.01,11023.80,20,3
225097,2021-03-04,Walker,Alabama,6585,259.00,1127,AL,County,63521,10366.65,407.74,407.74,10366.65,31,4
527393,2021-03-04,Crenshaw,Alabama,1462,55.00,1041,AL,County,13772,10615.74,399.36,399.36,10615.74,30,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567985,2021-03-04,Hot Springs,Wyoming,361,3.00,56017,WY,County,4413,8180.38,67.98,67.98,8180.38,17,19
779688,2021-03-04,Uinta,Wyoming,2070,12.00,56041,WY,County,20226,10234.35,59.33,59.33,10234.35,4,20
799512,2021-03-04,Lincoln,Wyoming,1244,11.00,56023,WY,County,19830,6273.32,55.47,55.47,6273.32,22,21
224044,2021-03-04,Teton,Wyoming,3368,9.00,56039,WY,County,23464,14353.90,38.36,38.36,14353.90,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3793.05,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3793.05,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3793.05,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3793.05,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3793.05,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039384,2021-02-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1039385,2021-03-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1039386,2021-03-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1039387,2021-03-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
731001,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,224.37,14562.70,39,1,2.00
731002,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,224.37,14562.70,39,1,0.00
731003,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,224.37,14562.70,39,1,1.00
731004,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,224.37,14562.70,39,1,4.00
731005,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,224.37,14562.70,39,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812796,2021-02-28,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5511.60,7,23,0.00
812797,2021-03-01,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5511.60,7,23,0.00
812798,2021-03-02,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,1.00
812799,2021-03-03,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
525965,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13643.84,1,3,1.00,0.00
525966,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13643.84,1,3,0.00,0.00
525967,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13643.84,1,3,0.00,0.00
525968,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13643.84,1,3,0.00,0.00
525969,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13643.84,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524240,2021-02-28,Albany,Wyoming,3905,11.00,56001,WY,County,38880,10043.72,28.29,28.29,10061.73,23,6,4.00,0.00
524241,2021-03-01,Albany,Wyoming,3909,11.00,56001,WY,County,38880,10054.01,28.29,28.29,10061.73,23,6,4.00,0.00
524242,2021-03-02,Albany,Wyoming,3911,11.00,56001,WY,County,38880,10059.16,28.29,28.29,10061.73,23,6,2.00,0.00
524243,2021-03-03,Albany,Wyoming,3911,11.00,56001,WY,County,38880,10059.16,28.29,28.29,10061.73,23,6,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-04') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
268589,2021-03-04,Imperial,California,26939,635.00,6025,CA,County,181215,14865.77,350.41,350.41,14865.77,1,2,13.00,0.00
2027,2021-03-04,Los Angeles,California,1198737,21778.00,6037,CA,County,10039107,11940.67,216.93,216.93,11940.67,2,5,2185.00,109.00
469734,2021-03-04,Inyo,California,1294,37.00,6027,CA,County,18039,7173.35,205.11,205.11,7173.35,3,27,2.00,0.00
60013,2021-03-04,Stanislaus,California,56522,955.00,6099,CA,County,550660,10264.41,173.43,173.43,10264.41,4,10,89.00,2.00
74370,2021-03-04,Tulare,California,48163,768.00,6107,CA,County,466195,10331.08,164.74,164.74,10331.08,5,9,49.00,0.00
27867,2021-03-04,Riverside,California,290744,3884.00,6065,CA,County,2470546,11768.41,157.21,157.21,11768.41,6,6,246.00,35.00
47775,2021-03-04,San Joaquin,California,67213,1177.00,6077,CA,County,762148,8818.89,154.43,154.43,8818.89,7,18,90.00,27.00
569300,2021-03-04,Kings,California,22119,227.00,6031,CA,County,152940,14462.53,148.42,148.42,14462.53,8,3,28.00,7.00
425809,2021-03-04,Merced,California,29325,412.00,6047,CA,County,277680,10560.72,148.37,148.37,10560.72,9,8,61.00,5.00
27141,2021-03-04,Fresno,California,95893,1458.00,6019,CA,County,999101,9597.93,145.93,145.93,9597.93,10,14,108.00,7.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
989962,2021-03-04,Lassen,California,5623,19.00,6035,CA,County,30573,18392.05,62.15,62.15,18392.05,38,1,0.00,0.00
268589,2021-03-04,Imperial,California,26939,635.00,6025,CA,County,181215,14865.77,350.41,350.41,14865.77,1,2,13.00,0.00
569300,2021-03-04,Kings,California,22119,227.00,6031,CA,County,152940,14462.53,148.42,148.42,14462.53,8,3,28.00,7.00
133965,2021-03-04,San Bernardino,California,287246,3022.00,6071,CA,County,2180085,13175.91,138.62,138.62,13175.91,11,4,191.00,63.00
2027,2021-03-04,Los Angeles,California,1198737,21778.00,6037,CA,County,10039107,11940.67,216.93,216.93,11940.67,2,5,2185.00,109.00
27867,2021-03-04,Riverside,California,290744,3884.00,6065,CA,County,2470546,11768.41,157.21,157.21,11768.41,6,6,246.00,35.00
168345,2021-03-04,Kern,California,103894,924.00,6029,CA,County,900202,11541.19,102.64,102.64,11541.19,18,7,138.00,9.00
425809,2021-03-04,Merced,California,29325,412.00,6047,CA,County,277680,10560.72,148.37,148.37,10560.72,9,8,61.00,5.00
74370,2021-03-04,Tulare,California,48163,768.00,6107,CA,County,466195,10331.08,164.74,164.74,10331.08,5,9,49.00,0.00
60013,2021-03-04,Stanislaus,California,56522,955.00,6099,CA,County,550660,10264.41,173.43,173.43,10264.41,4,10,89.00,2.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
268589,2021-03-04,Imperial,California,26939,635.00,6025,CA,County,181215,14865.77,350.41,350.41,14865.77,1,2,13.00,0.00
2027,2021-03-04,Los Angeles,California,1198737,21778.00,6037,CA,County,10039107,11940.67,216.93,216.93,11940.67,2,5,2185.00,109.00
469734,2021-03-04,Inyo,California,1294,37.00,6027,CA,County,18039,7173.35,205.11,205.11,7173.35,3,27,2.00,0.00
60013,2021-03-04,Stanislaus,California,56522,955.00,6099,CA,County,550660,10264.41,173.43,173.43,10264.41,4,10,89.00,2.00
74370,2021-03-04,Tulare,California,48163,768.00,6107,CA,County,466195,10331.08,164.74,164.74,10331.08,5,9,49.00,0.00
27867,2021-03-04,Riverside,California,290744,3884.00,6065,CA,County,2470546,11768.41,157.21,157.21,11768.41,6,6,246.00,35.00
47775,2021-03-04,San Joaquin,California,67213,1177.00,6077,CA,County,762148,8818.89,154.43,154.43,8818.89,7,18,90.00,27.00
569300,2021-03-04,Kings,California,22119,227.00,6031,CA,County,152940,14462.53,148.42,148.42,14462.53,8,3,28.00,7.00
425809,2021-03-04,Merced,California,29325,412.00,6047,CA,County,277680,10560.72,148.37,148.37,10560.72,9,8,61.00,5.00
27141,2021-03-04,Fresno,California,95893,1458.00,6019,CA,County,999101,9597.93,145.93,145.93,9597.93,10,14,108.00,7.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,350.41,14865.77,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,350.41,14865.77,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,350.41,14865.77,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,350.41,14865.77,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,350.41,14865.77,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3512,02/28/21,Lassen,5601,19.00,18320.09,62.15,62.15,18392.05,38,1,0.00,0.00
3513,03/01/21,Lassen,5601,19.00,18320.09,62.15,62.15,18392.05,38,1,0.00,0.00
3514,03/02/21,Lassen,5601,19.00,18320.09,62.15,62.15,18392.05,38,1,0.00,0.00
3515,03/03/21,Lassen,5623,19.00,18392.05,62.15,62.15,18392.05,38,1,22.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,350.41,14865.77,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,350.41,14865.77,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,350.41,14865.77,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,350.41,14865.77,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,350.41,14865.77,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3586,02/28/21,Fresno,95393,1443.00,9547.88,144.43,145.93,9597.93,10,14,191.00,0.00
3587,03/01/21,Fresno,95548,1443.00,9563.40,144.43,145.93,9597.93,10,14,155.00,0.00
3588,03/02/21,Fresno,95677,1451.00,9576.31,145.23,145.93,9597.93,10,14,129.00,8.00
3589,03/03/21,Fresno,95785,1451.00,9587.12,145.23,145.93,9597.93,10,14,108.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)